In [1]:
!pip install anomalib
!pip install lightning kornia torchmetrics torch FrEIA python-dotenv open-clip-torch
!pip install qdrant-client
# Cell 2: Import required libraries
from anomalib.data import MVTec
from anomalib.engine import Engine
from anomalib.models import Patchcore, EfficientAd
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm.notebook import tqdm
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from PIL import Image
import torchvision.transforms as T

  Using cached lightning-2.4.0-py3-none-any.whl.metadata (38 kB)
  Using cached kornia-0.7.3-py2.py3-none-any.whl.metadata (7.7 kB)
  Using cached torchmetrics-1.5.1-py3-none-any.whl.metadata (20 kB)
  Using cached FrEIA-0.2.tar.gz (34 kB)
  Preparing metadata (setup.py) ... done
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached open_clip_torch-2.29.0-py3-none-any.whl.metadata (31 kB)
  Using cached pytorch_lightning-2.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached kornia_rs-0.1.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.1 kB)
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
Using cached lightning-2.4.0-py3-none-any.whl (810 kB)
Using cached kornia-0.7.3-py2.py3-none-any.whl (833 kB)
Using cached torchmetrics-1.5.1-py3-none-any.whl (890 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached open_clip_torch-2.29.0-py3-none-any.whl (1.5 MB)
Using cached kornia_rs-0.1.7-cp310-cp310-manyli

/usr/local/lib/python3.10/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [12]:
def train_and_evaluate(category, model_class=EfficientAd):
    """Train and evaluate a model on a specific category."""
    print(f"Training {model_class.__name__} on {category}")

    batch_size = 1 if model_class == EfficientAd else 32
    datamodule = MVTec(
        root="./datasets/MVTec",
        category=category,
        train_batch_size=batch_size
    )
    model = model_class()
    engine = Engine(max_epochs=5)

    engine.fit(datamodule=datamodule, model=model)
    results = engine.test(model, datamodule)

    return model, datamodule, {
        'image_AUROC': results[0]['image_AUROC'],
        'image_F1Score': results[0]['image_F1Score'],
        'pixel_AUROC': results[0]['pixel_AUROC'],
        'pixel_F1Score': results[0]['pixel_F1Score']
    }

# Fixed Visualization Function
def visualize_sample(image, label, prediction, anomaly_map):
    """Visualize a single sample."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Original
    axes[0].imshow(image)
    axes[0].set_title(f'Original (Label: {"Anomaly" if label else "Normal"})')
    axes[0].axis('off')

    # Anomaly Map
    im = axes[1].imshow(anomaly_map, cmap='jet')
    axes[1].set_title(f'Anomaly Score: {prediction:.3f}')
    plt.colorbar(im, ax=axes[1])
    axes[1].axis('off')

    # Overlay
    axes[2].imshow(image)
    axes[2].imshow(anomaly_map, cmap='jet', alpha=0.5)
    axes[2].set_title('Overlay')
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

def visualize_predictions(model, datamodule):
    """Visualize model predictions."""
    test_loader = datamodule.test_dataloader()
    batch = next(iter(test_loader))

    images, labels = batch[0], batch[1]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Process images one at a time
    for i in range(3):  # Show 3 samples
        image = images[i:i+1].to(device)
        label = labels[i]

        model.eval()
        with torch.no_grad():
            prediction = model(image)
            anomaly_map = model.anomaly_map(image)

        # Convert tensors to numpy for visualization
        image_np = image[0].cpu().permute(1, 2, 0).numpy()
        prediction_np = prediction[0].cpu().item()
        anomaly_map_np = anomaly_map[0].cpu().squeeze().numpy()

        visualize_sample(image_np, label, prediction_np, anomaly_map_np)

# Plot Metrics Function
def plot_results(results):
    """Plot all metrics."""
    metrics = ['image_AUROC', 'image_F1Score', 'pixel_AUROC', 'pixel_F1Score']
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()

    for idx, metric in enumerate(metrics):
        scores = []
        labels = []

        for model in results:
            for category in results[model]:
                scores.append(results[model][category][metric])
                labels.append(f"{model}\n{category}")

        axes[idx].bar(range(len(scores)), scores)
        axes[idx].set_xticks(range(len(scores)))
        axes[idx].set_xticklabels(labels, rotation=45)
        axes[idx].set_title(metric)
        axes[idx].set_ylim(0, 1)

    plt.tight_layout()
    plt.show()


def run_analysis(categories=['tile', 'leather', 'grid']):
    """Run analysis on specified categories."""
    results = {
        'EfficientAd': {},
        'PatchCore': {}
    }

    for category in categories:
        print(f"\nProcessing {category}")

        try:
            # Train and evaluate EfficientAD
            model, datamodule, metrics = train_and_evaluate(category, EfficientAd)
            results['EfficientAd'][category] = metrics

            print(f"\nEfficientAD metrics for {category}:")
            for metric, value in metrics.items():
                print(f"{metric}: {value:.4f}")

            print("\nVisualizing EfficientAD predictions:")
            visualize_predictions(model, datamodule)
        except Exception as e:
            print(f"Error with EfficientAD on {category}: {str(e)}")

        try:
            # Train and evaluate PatchCore
            model, datamodule, metrics = train_and_evaluate(category, Patchcore)
            results['PatchCore'][category] = metrics

            print(f"\nPatchCore metrics for {category}:")
            for metric, value in metrics.items():
                print(f"{metric}: {value:.4f}")

            print("\nVisualizing PatchCore predictions:")
            visualize_predictions(model, datamodule)
        except Exception as e:
            print(f"Error with PatchCore on {category}: {str(e)}")

    return results

# Run Analysis
if __name__ == "__main__":
    # Run the analysis
    results = run_analysis()

    # Plot results
    plot_results(results)

    # Print summary
    print("\nFinal Results Summary:")
    metrics = ['image_AUROC', 'image_F1Score', 'pixel_AUROC', 'pixel_F1Score']

    for model in results:
        print(f"\n{model}:")
        for metric in metrics:
            scores = [results[model][cat][metric] for cat in results[model]]
            avg = np.mean(scores)
            print(f"Average {metric}: {avg:.4f}")


Processing tile
Training EfficientAd on tile


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
mvtec: 5.26GB [04:37, 19.0MB/s]                            
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if nec

Training: |          | 0/? [00:00<?, ?it/s]


efficientad_pretrained_weights.zip: 0.00B [00:00, ?B/s]
efficientad_pretrained_weights.zip:   0%|          | 8.19k/40.0M [00:01<1:51:13, 5.99kB/s]
efficientad_pretrained_weights.zip: 40.0MB [00:01, 26.0MB/s]                            
/usr/local/lib/python3.10/dist-packages/anomalib/models/image/efficient_ad/lightning_model.py:98: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend yo

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]



Selecting Coreset Indices.:   0%|          | 0/20697 [00:00<?, ?it/s]

Selecting Coreset Indices.:   0%|          | 9/20697 [00:00<04:05, 84.38it/s]

Selecting Coreset Indices.:   0%|          | 18/20697 [00:00<04:54, 70.30it/s]

Selecting Coreset Indices.:   0%|          | 26/20697 [00:00<04:58, 69.22it/s]

Selecting Coreset Indices.:   0%|          | 35/20697 [00:00<04:41, 73.35it/s]

Selecting Coreset Indices.:   0%|          | 43/20697 [00:00<04:38, 74.07it/s]

Selecting Coreset Indices.:   0%|          | 51/20697 [00:00<04:47, 71.93it/s]

Selecting Coreset Indices.:   0%|          | 59/20697 [00:00<04:40, 73.51it/s]

Selecting Coreset Indices.:   0%|          | 68/20697 [00:00<04:26, 77.32it/s]

Selecting Coreset Indices.:   0%|          | 81/20697 [00:01<03:43, 92.08it/s]

Selecting Coreset Indices.:   0%|          | 92/20697 [00:01<03:36, 94.96it/s]

Selecting Coreset Indices.:   0%|          | 102/20697 [00:01<03:47, 90.71it/s]

Selecting Coreset Indices.:   1%|          | 11

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.9766082167625427     │
│       image_F1Score       │    0.9557521939277649     │
│        pixel_AUROC        │    0.9789097905158997     │
│       pixel_F1Score       │    0.37611815333366394    │
└───────────────────────────┴───────────────────────────┘


PatchCore metrics for grid:
image_AUROC: 0.9766
image_F1Score: 0.9558
pixel_AUROC: 0.9789
pixel_F1Score: 0.3761

Visualizing PatchCore predictions:
Error with PatchCore on grid: 0

Final Results Summary:

EfficientAd:
Average image_AUROC: 0.9472
Average image_F1Score: 0.9426
Average pixel_AUROC: 0.8986
Average pixel_F1Score: 0.4819

PatchCore:
Average image_AUROC: 0.9881
Average image_F1Score: 0.9774
Average pixel_AUROC: 0.9722
Average pixel_F1Score: 0.4788
